In [29]:
from google.cloud import bigquery
from BigQueryHelper import BigQueryHelper as bqh
import os
import pandas as pd
import json

In [30]:
pathToRoot = '../../../'
client = bigquery.Client.from_service_account_json(os.path.join(pathToRoot, 'credentials/rescribe-nlp-creds.txt'))

In [31]:
data = bqh(active_project="bigquery-public-data",
            dataset_name="stackoverflow", 
            client=client)

data.list_tables()

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [32]:
#Ok, so what we want are the query titles, the query descriptions, the code from the top answer, and the tags associated with it
#Note: individual tables in this dataset blow past 30 gigs

QUERY = """
SELECT 
    id, title, body, accepted_answer_id
FROM 
    bigquery-public-data.stackoverflow.posts_questions
WHERE accepted_answer_id IS NOT NULL
LIMIT 10000

"""
questions = data.query_to_pandas(QUERY)

In [33]:
print(f"Number of rows: {len(questions)}")
questions.sample(5)

Number of rows: 10000


,id,title,body,accepted_answer_id
4085,61990203,javascript dynamic filters with OR results,<p>I have an array of objects with multiple fi...,61990316
3689,61814697,Queryset for ManyToMany relation ordered by nu...,"<p>I have two models, Entity and Tag with a Ma...",61816560
7256,61789308,Best practice to delete old MetricBeats data i...,<p>How do we best delete old data collected vi...,61790045
3099,61837435,Call to a member function checkLearning() on n...,<p>hello i use this method but not worked on v...,61838050
7874,62047149,Unable to use custom domain on localhost using...,<p>I want to use a custom domain for my larave...,62047383


In [34]:
with open(os.path.join(pathToRoot,"datasets/post-questions.json"), 'w') as outfile: 
    json.dump(questions.to_json(), outfile)

In [35]:
#alright, time for the answers
QUERY = f"""
SELECT 
    id, body
FROM 
    bigquery-public-data.stackoverflow.posts_answers
WHERE id IN {tuple(questions.accepted_answer_id.values)}
LIMIT 10000
"""

answers = data.query_to_pandas(QUERY)

In [36]:
print(f"Number of rows: {len(answers)}")
answers.sample(5)

Number of rows: 10000


,id,body
6268,61956718,<p>@pdeuxa you need to configure the SSLContex...
5064,61973028,<p>I'm not 100% sure how Process works in this...
6172,61802282,<p>This is <em>not a Docusaurus issue</em>.</p...
6479,61812034,"<p>Yes, they are. According to <a href=""https:..."
2921,61987155,<p>You can set <code>background</code> propert...


In [ ]:
with open(os.path.join(pathToRoot,"datasets/post-answers.json"), 'w') as outfile: 
    json.dump(answers.to_json(), outfile)